In [102]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import reset_ramsey
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [103]:
source_file = 'Electricity.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,cost,q,pl,sl,pk,sk,pf,sf
0,0.2130,8.0,6869.47,0.3291,64.945,0.4197,18.000,0.2512
1,3.0427,869.0,8372.96,0.1030,68.227,0.2913,21.067,0.6057
2,9.4059,1412.0,7960.90,0.0891,40.692,0.1567,41.530,0.7542
3,0.7606,65.0,8971.89,0.2802,41.243,0.1282,28.539,0.5916
4,2.2587,295.0,8218.40,0.1772,71.940,0.1623,39.200,0.6606
...,...,...,...,...,...,...,...,...
153,6.8293,946.6,10642.16,0.0883,43.600,0.1914,51.463,0.7203
154,3.7605,377.0,7432.24,0.2117,74.120,0.2274,33.436,0.5609
155,3.9822,391.0,5826.04,0.1926,78.288,0.0924,44.633,0.7151
156,30.1880,5317.0,9586.63,0.0845,78.008,0.2009,41.840,0.7147


### 2 Сравнение моделей
#### (Вариант 2)

#### 2.3 cost equations

Для набора данных `Electricity` рассмотрим несколько моделей регрессии.



In [104]:
mod = []

In [105]:
formula = 'np.log(cost) ~ np.log(q) + np.log(pl) + np.log(pk) + np.log(pf)'
mod.append(smf.ols(formula = formula, data = df).fit())
formula2 = formula + ' + I(np.log(q)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' - np.log(q) + q'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' - np.log(q) + q + I(q**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(np.log(q)**2) + I(np.log(pl)**2)+I(np.log(pk)**2)+I(np.log(pf)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' - np.log(q) + q + I(q**2) + I(np.log(pl)**2) + I(np.log(pk)**2) + I(np.log(pf)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())

In [106]:

print(
    summary_col(
        mod,
        float_format='%.3f',
        stars=True,
        model_names=['1', '2', '3', '4', '5', '6']
        # info_dict = info_dict
    )
)


                       1         2         3         4        5         6    
-----------------------------------------------------------------------------
I(np.log(pf) ** 2)                                         -0.031   -0.127   
                                                           (0.093)  (0.597)  
I(np.log(pk) ** 2)                                         -0.036   -0.470   
                                                           (0.187)  (1.202)  
I(np.log(pl) ** 2)                                         -0.455   -2.785   
                                                           (0.297)  (1.909)  
I(np.log(q) ** 2)            0.030***                      0.031***          
                             (0.002)                       (0.002)           
I(q ** 2)                                        -0.000***          -0.000***
                                                 (0.000)            (0.000)  
Intercept          -7.472*** -6.739*** -12.226** -10.876** -44.

In [107]:
df = pd.DataFrame()
for ind, model in enumerate(mod):
    df['mod' + str(ind+1)] = [
        model.rsquared_adj.round(3),
        model.aic.round(3),
        model.bic.round(3)
    ]
df

,mod1,mod2,mod3,mod4,mod5,mod6
0,0.982,0.992,0.498,0.667,0.992,0.666
1,-41.855,-171.030,481.433,417.566,-167.985,420.654
2,-26.542,-152.654,496.746,435.942,-140.422,448.217


In [108]:
df_res = pd.DataFrame(columns=['Метод', 'Регрессия'])
df_res.loc[0] = ['Adj.R2', df.loc[0].idxmax()]
df_res.loc[1] = ['AIC', df.loc[1].idxmin()]
df_res.loc[2] = ['BIC', df.loc[2].idxmin()]
df_res

,Метод,Регрессия
0,Adj.R2,mod2
1,AIC,mod2
2,BIC,mod2
